In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - Training formalization

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?&download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/05_model_training_formalization.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/05_model_training_formalization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/05_model_training_formalization.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to get your training dataset from Vertex AI Feature Store, train your model using Vertex AI managed training pipeline, and deploy it as a Vertex AI endpoint. You will learn how to use your own custom code for ML training on Vertex AI.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

* build a container to run your own custom code on Vertex AI
* use Vertex AI to train your model at scale
* use Vertex AI to create an endpoint

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.### Load config settings

In [63]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-vision-workshop"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "temp-vision-workshop-vision-workshop"
PROJECT              = "temp-vision-workshop"
REGION               = "europe-west4"
ID                   = "7l3oe"
MODEL_NAME           = "vision_workshop_model"
ENDPOINT_NAME        = "vision_workshop_endpoint"



In [64]:
from datetime import datetime, timedelta

END_DATE_TRAIN = datetime.today().strftime("%Y-%m-%d")

## Custom Training
DATASET_NAME=f"sample_train-{ID}-{END_DATE_TRAIN}"
TRAIN_JOB_NAME=f"vision_train_frmlz-{ID}"
MODEL_NAME=f"vision_model_frmlz-{ID}"
DEPLOYED_NAME = f"vision_prediction_frmlz-{ID}"
MODEL_SERVING_IMAGE_URI = "europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest"
IMAGE_REPOSITORY = f"vision-{ID}"
IMAGE_NAME="image-classifier"
IMAGE_TAG="v1"
IMAGE_URI=f"europe-west4-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"
TRAIN_COMPUTE="e2-standard-4"
DEPLOY_COMPUTE="n1-standard-4"

In [65]:
IMAGE_REPOSITORY

'vision-7l3oe'



## Builing a custom fraud detection model

### Fixing an imbalanced dataset
In the real world, we need to deal with imbalance in our dataset. For example, we might randomly delete some of the non-fraudulent transcations in order to approximately match the number of fraudulent transactions. This technique is called undersampling. 

For this workshop, we will skip the data balance process, because our sample data is small and further reduction will compromise the quality of our results. If you have a larger sample and would like to balance your data, you can uncomment and run the following cell.

In [54]:
# from urllib.parse import urlparse
# obj_list = gcs_list(TRAIN_DATA_URI)

# TRAIN_DATA_URI_BALANCED = f"{DATA_URI}/balanced_train"
# TRAIN_DATA_DIR_BALANCED = os.path.join(DATA_DIR, "balanced_train")
# if not os.path.exists(TRAIN_DATA_DIR_BALANCED):
#     os.makedirs(TRAIN_DATA_DIR_BALANCED)
# for ds_csv_uri in obj_list:
#     tx_df = pd.read_csv(ds_csv_uri)
#     shuffled_df = tx_df.sample(frac=1,random_state=4)
#     fraud_df = shuffled_df.loc[shuffled_df["tx_fraud"] == 1]
#     non_fraud_df = shuffled_df.loc[shuffled_df["tx_fraud"] == 0].sample(n=fraud_df.shape[0],random_state=42)
#     balanced_df = pd.concat([fraud_df, non_fraud_df])
#     balanced_df.to_csv(os.path.join(TRAIN_DATA_DIR_BALANCED,  os.path.basename(urlparse(ds_csv_uri).path)), index=False)
#     break

# !gsutil cp -r  $TRAIN_DATA_DIR_BALANCED $TRAIN_DATA_URI_BALANCED
# obj_list = gcs_list(TRAIN_DATA_URI_BALANCED)

#### Builing a Vertex AI dataset
In this section, we will build a Vertex AI dataset from our tabular data. Vertex AI datasets can be used to train AutoML models or custom-trained models.  

In [ ]:
dataset = create_gcs_dataset(client=vertex_ai, display_name=DATASET_NAME, gcs_source=obj_list[0]) #obj_list

print("Dataset:", f"{dataset.display_name}")
print("Name: \t", f"{dataset.resource_name}")

In [ ]:
from google.cloud import storage

client = storage.Client() 

# Implicit environment set up
# with explicit set up:
# client = storage.Client.from_service_account_json('key-file-location')

blobs = list(client.list_blobs(BUCKET_NAME, prefix='flowers/'))

In [ ]:
labels = [os.path.split(os.path.dirname(blob.name))[1] for blob in blobs]

In [ ]:
d = [[f"gs://{blob.bucket.name}/{blob.name}", os.path.split(os.path.dirname(blob.name))[1]] for blob in blobs]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(d)
df

In [ ]:
df.to_csv(f"gs://{BUCKET_NAME}/flowers/flowers.csv",index=False, header=False)

In [ ]:
ds = vertex_ai.ImageDataset.create(
    display_name="flowers",
    gcs_source=f"gs://{BUCKET_NAME}/flowers/flowers.csv",
    import_schema_uri=vertex_ai.schema.dataset.ioformat.image.single_label_classification,
    sync=True,
)

ds.wait()

print(ds.display_name)
print(ds.resource_name)

In [ ]:
ds = ImageDataset.list(filter="display_name=flowers")[0]

### Train a custom model
In this section, we will use the xgboost algorithm. Specifically, we will perform custom training with a pre-built xgboost container.

#### Create the training application
Typically, to perform custom training you can use either a pre-built container or buid your own container. In this section we will build a container for xgboost, and use it to train a model with the Vertex AI Managed Training service.

The first step is to write your training code. Then, write a Dockerfile and build a container image based on it. The following cell, writes our code into `train_gb.py` which is the module for training a XGBClassifier. We will copy this code later into our container to be run through Vertex Training service.

In [66]:
!mkdir -p -m 777 build_training

In [67]:
%%writefile build_training/train_tf.py

"""
train_gb.py is the module for training a XGBClassifier pipeline
"""

# Libraries --------------------------------------------------------------------------------------------------------------------------
import argparse
import json
import logging
from pathlib import Path
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import time
from datetime import datetime, timedelta
from google.cloud import aiplatform as vertex_ai
from pathlib import Path

# Variables --------------------------------------------------------------------------------------------------------------------------
## Read environmental variables
TRAINING_DATA_PATH = os.environ["AIP_TRAINING_DATA_URI"].replace("gs://", "/gcs/")
TEST_DATA_PATH = os.environ["AIP_TEST_DATA_URI"].replace("gs://", "/gcs/")
MODEL_DIR = os.environ["AIP_MODEL_DIR"].replace("gs://", "/gcs/")

# Helpers -----------------------------------------------------------------------------------------------------------------------------
def get_args():
    parser = argparse.ArgumentParser()

    # Data files arguments
    parser.add_argument("--data_dir", dest="data_dir", type=str,
                        required=True, help="Bucket uri")
    parser.add_argument("--lr", dest="lr",
                        default=6, type=float,
                        help="max_depth value.")
    parser.add_argument("-v", "--verbose", 
                        help="increase output verbosity", 
                        action="store_true")
    
    return parser.parse_args()

def set_logging():
    #TODO
    pass

# def evaluate_model(model, x_true, y_true):
    
#     y_true = y_true.compute()
    
#     #calculate metrics
#     metrics={}
    
#     y_score =  model.predict_proba(x_true)[:, 1]
#     y_score = y_score.compute()
#     fpr, tpr, thr = roc_curve(
#          y_true=y_true, y_score=y_score, pos_label=True
#     )
#     fpr_list = fpr.tolist()[::50]
#     tpr_list = tpr.tolist()[::50]
#     thr_list = thr.tolist()[::50]

#     y_pred = model.predict(x_true)
#     y_pred.compute()
#     c_matrix = confusion_matrix(y_true, y_pred)
    
#     avg_precision_score = round(average_precision_score(y_true, y_score), 3)
#     f1 = round(f1_score(y_true, y_pred), 3)
#     lg_loss = round(log_loss(y_true, y_pred), 3)
#     prec_score = round(precision_score(y_true, y_pred), 3)
#     rec_score = round(recall_score(y_true, y_pred), 3)
    
    
#     metrics["fpr"] = [round(f, 3) for f in fpr_list]
#     metrics["tpr"] = [round(f, 3) for f in tpr_list]
#     metrics["thrs"] = [round(f, 3) for f in thr_list]
#     metrics["confusion_matrix"] = c_matrix.tolist()
#     metrics["avg_precision_score"] = avg_precision_score
#     metrics["f1_score"] = f1
#     metrics["log_loss"] = lg_loss
#     metrics["precision_score"] = prec_score
#     metrics["recall_score"] = rec_score
    
#     return metrics


def main():
    args = get_args()
    if args.verbose:
        set_logging()
        
    #variables
    data_dir = args.data_dir.replace("gs://", "/gcs/")
    data_dir = Path(data_dir)

    #read data
    print(f"TRAINING_DATA_PATH: {TRAINING_DATA_PATH}")
    batch_size = 16
    img_height = 384
    img_width = 384
    
    train_ds = tf.keras.utils.image_dataset_from_directory(
      data_dir,
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    #train model
    base_model = hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2", trainable=False)
    # Create new model on top
    inputs = tf.keras.Input(shape=(img_height, img_width, 3))
    data_augmentation = tf.keras.Sequential(
        [
            tf.keras.layers.RandomFlip("horizontal"), 
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomTranslation(0, 0.2),
            tf.keras.layers.RandomTranslation(0.2, 0),
            tf.keras.layers.RandomZoom(0.2, 0.2),
        ]
    )
    x = data_augmentation(inputs)  # Apply random data augmentation
    x = tf.keras.layers.Rescaling(1./255)(x)
    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = base_model(x)
    #x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = tf.keras.layers.Dense(5)(x)
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.0001)
    history = model.fit(
      train_ds,
      epochs=20, 
      callbacks=[reduce_lr]
    )
    
    if not Path(MODEL_DIR).exists():
        Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)
    model.save(MODEL_DIR)
    
    #generate metrics
    # metrics = evaluate_model(model, x_true, y_true)
    # if not Path(deliverable_uri).exists():
    #     Path(deliverable_uri).mkdir(parents=True, exist_ok=True)
    # with open(metrics_uri, "w") as file:
    #     json.dump(metrics, file, sort_keys = True, indent = 4)
    # file.close()
    
if __name__ == "__main__":
    main()

Overwriting build_training/train_tf.py


#### Define a custom image for dask model training

Here we will build a custom container. A custom container is a Docker image that you create to run your training application. By running your machine learning (ML) training job in a custom container, you can use ML frameworks, non-ML dependencies, libraries, and binaries that are not otherwise supported on Vertex AI. In othere word, we package training code on our local machine into a Docker container image, push the container image to Container Registry, and create a CustomJob.

For the ML framework we will use xgboost. We also use dask and scikit libraries. Dask is an open source library for parallel computing written in Python. We will use Dask to speed up pre-processing of our dataset.

In [68]:
# Create image repo
!gcloud artifacts repositories create $IMAGE_REPOSITORY \
    --repository-format=docker \
    --location=europe-west4 \
    --description="Vision Workshop Docker Image repository"

# List repositories under the project
!gcloud artifacts repositories list

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists
Listing items under project temp-vision-workshop, across all locations.

                                                                                ARTIFACT_REGISTRY
REPOSITORY    FORMAT  MODE                 DESCRIPTION                              LOCATION      LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
vision-7l3oe  DOCKER  STANDARD_REPOSITORY  Vision Workshop Docker Image repository  europe-west4          Google-managed key  2022-11-07T18:05:03  2022-11-08T09:39:25  1581.483


In [69]:
!gcloud auth configure-docker europe-west4-docker.pkg.dev -q


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "europe-west4-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: europe-west4-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [70]:
%%writefile build_training/Dockerfile
# Specifies base image and tag
# TO DO: to change it to python-slim
FROM python:3.7 
WORKDIR /root

# Installs additional packages
RUN pip install gcsfs numpy pandas scikit-learn tensorflow==2.8.3 tensorflow_datasets tensorflow_hub google-cloud-aiplatform --upgrade

# Copies the trainer code to the docker image.
COPY ./train_tf.py /root/train_tf.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "train_tf.py"]

Overwriting build_training/Dockerfile


In [71]:
# Build and push docker file
!docker build -t $IMAGE_URI ./build_training/
!docker push $IMAGE_URI

Sending build context to Docker daemon  8.192kB
Step 1/5 : FROM python:3.7
 ---> c643d609e965
Step 2/5 : WORKDIR /root
 ---> Using cache
 ---> 52a8a881f1bd
Step 3/5 : RUN pip install gcsfs numpy pandas scikit-learn tensorflow==2.8.3 tensorflow_datasets tensorflow_hub google-cloud-aiplatform --upgrade
 ---> Using cache
 ---> 4546d8d64b09
Step 4/5 : COPY ./train_tf.py /root/train_tf.py
 ---> 06d45e886550
Step 5/5 : ENTRYPOINT ["python3", "train_tf.py"]
 ---> Running in a9a9dc9c224f
Removing intermediate container a9a9dc9c224f
 ---> 1319b167ba62
Successfully built 1319b167ba62
Successfully tagged europe-west4-docker.pkg.dev/temp-vision-workshop/vision-7l3oe/image-classifier:v1
The push refers to repository [europe-west4-docker.pkg.dev/temp-vision-workshop/vision-7l3oe/image-classifier]

5e0fd0a4: Preparing 
c06c4dfc: Preparing 
f78a199d: Preparing 
c04719be: Preparing 
8b761eaf: Preparing 
3c62e3d7: Preparing 
d36bfd35: Preparing 
c9917839: Preparing 
df39e1dd: Preparing 
21b04368: Prepar

#### Submit the script to run on Vertex AI
In this section, we create a training pipeline. It will create custom training jobs, load our dataset and upload the model to Vertex AI after the training job is successfully completed. Learn more about creating of custom jobs [here](https://cloud.google.com/vertex-ai/docs/training/create-custom-job).

In [72]:
from google.cloud import aiplatform as vertex_ai


vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME, experiment="train")


ds = vertex_ai.ImageDataset.list(filter="display_name=flowers")[0]
ds

resource name: projects/446303513828/locations/europe-west4/datasets/4893719547044954112

In [ ]:
job = vertex_ai.CustomContainerTrainingJob(
    display_name=TRAIN_JOB_NAME,
    container_uri=IMAGE_URI,
    model_serving_container_image_uri=MODEL_SERVING_IMAGE_URI,
)

parameters = {"LR": 0.003}

CMDARGS = [ f"""--data_dir=gs://{BUCKET_NAME}/flowers""",
    "--lr=" + str(parameters["LR"]),
    "--verbose"
]


model = job.run(
    dataset=ds,
    annotation_schema_uri=vertex_ai.schema.dataset.annotation.image.classification,
    model_display_name=MODEL_NAME,
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_count=0)

Training Output directory:
gs://temp-vision-workshop-vision-workshop/aiplatform-custom-training-2022-11-08-10:24:52.496 
No dataset split provided. The service will use a default split.
View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/2086418971216576512?project=446303513828
CustomContainerTrainingJob projects/446303513828/locations/europe-west4/trainingPipelines/2086418971216576512 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/5948466757919309824?project=446303513828
CustomContainerTrainingJob projects/446303513828/locations/europe-west4/trainingPipelines/2086418971216576512 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/446303513828/locations/europe-west4/trainingPipelines/2086418971216576512 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/446303513828/loc

#### Evaluate the model locally

Before you can run the model via an endpoint, you need to preprocess it to match the format that your custom model defined in task.py expects.

In [ ]:
LABEL_COLUMN = "tx_fraud"
UNUSED_COLUMNS = ["timestamp","entity_type_customer","entity_type_terminal"]
NA_VALUES = ["NA", "."]
def preprocess(df):
    """Converts categorical features to numeric. Removes unused columns.

    Args:
      df: Pandas df with raw data

    Returns:
      df with preprocessed data
    """
    df = df.drop(columns=UNUSED_COLUMNS)

    # Drop rows with NaN's
    df = df.dropna()

    # Convert integer valued (numeric) columns to floating point
    numeric_columns = df.select_dtypes(["int32", "float32", "float64"]).columns
    df[numeric_columns] = df[numeric_columns].astype("float32")

    dummy_columns = list(df.dtypes[df.dtypes == "category"].index)
    df = pd.get_dummies(df, columns=dummy_columns)

    return df
#test set
train_sample_path = os.path.join(TRAIN_DATA_DIR, "000000000000.csv")
df_test = pd.read_csv(train_sample_path)
preprocessed_test_Data = preprocess(df_test)

x_test = preprocessed_test_Data[preprocessed_test_Data.columns.drop(LABEL_COLUMN).to_list()].values
y_test = preprocessed_test_Data.loc[:,LABEL_COLUMN].astype(int)

Here we copy the model artifact to the local directory to evaluate the model localy before deploying the model:

In [ ]:
!gsutil cp -r $model.uri .

In the folliwng cell, we use the test dataset, to predict the accuracy of the model:

In [ ]:
bst = xgb.Booster()  # init model
bst.load_model("./model/model.bst") 
xgtest = xgb.DMatrix(x_test)
y_pred_prob = bst.predict(xgtest)
y_pred = y_pred_prob.round().astype(int)
y_pred_prob[0:10]
precision_recall_fscore_support(y_test.values, y_pred, average="weighted")

#### Deploy the model
Before you use your model to make predictions, you need to deploy it to an Endpoint. You can do this by calling the deploy function on the Model resource. This will do two things:

- create an Endpoint resource
- deploy the Model resource to the Endpoint resource

TO DO: PRE-DEPLOYING THIS MODEL SO USER DOESN'T HAVE TO WAIT

In [ ]:
DEPLOY_COMPUTE="n1-standard-4"
TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1


endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    accelerator_count=0,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

#### Test the deployed model (Make an online prediction request)
Send an online prediction request to your deployed model. To make sure your deployed model is working, test it out by sending a request to the endpoint.

Let's first get a test data.

In [ ]:
payload = {
  "instances": x_test[:2].tolist()
  
}

# In case you want to test it in the console
import json
with open("predictions.json", "w", encoding="utf-8") as f:
    json.dump(payload, f, ensure_ascii=False, indent=4)

In [ ]:
endpoint.predict(instances = payload["instances"])

## (DO NOT RUN) Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:



In [ ]:
# Delete endpoint resource
#! gcloud ai endpoints delete $ENDPOINT_NAME --quiet --region $REGION_NAME

# Delete model resource
#! gcloud ai models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
#! gsutil -m rm -r $JOB_DIR